In [1]:
import os; os.chdir('../')

In [2]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import BinaryLogitStep
import orca

/Users/maurer/anaconda3/envs/ual-model/lib/python3.6/site-packages/statsmodels-0.8.0-py3.6-macosx-10.9-x86_64.egg/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

In [4]:
orca.list_tables()

['households', 'buildings']

In [6]:
type(orca.get_table('households').to_frame())

pandas.core.frame.DataFrame

In [7]:
for table_name in orca.list_tables():
    print(table_name.upper())
    print(orca.get_table(table_name).to_frame().columns.tolist())
    print()

HOUSEHOLDS
['household_id', 'taz', 'serialno', 'puma5', 'income', 'persons', 'hht', 'unittype', 'noc', 'bldgsz', 'tenure', 'vehicl', 'hinccat1', 'hinccat2', 'hhagecat', 'hsizecat', 'hfamily', 'hunittype', 'hnoccat', 'hwrkrcat', 'h0004', 'h0511', 'h1215', 'h1617', 'h1824', 'h2534', 'h3549', 'h5064', 'h6579', 'h80up', 'hworkers', 'hwork_f', 'hwork_p', 'huniv', 'hnwork', 'hretire', 'hpresch', 'hschpred', 'hschdriv', 'htypdwel', 'hownrent', 'hadnwst', 'hadwpst', 'hadkids', 'bucketbin', 'originalpuma', 'hmultiunit', 'building_id']

BUILDINGS
['building_id', 'parcel_id', 'development_type_id', 'improvement_value', 'residential_units', 'residential_sqft', 'sqft_per_unit', 'non_residential_sqft', 'building_sqft', 'nonres_rent_per_sqft', 'res_price_per_sqft', 'stories', 'year_built', 'redfin_sale_price', 'redfin_sale_year', 'redfin_home_type', 'costar_property_type', 'costar_rent', 'building_type_id']



In [15]:
h = orca.get_table('households').to_frame()

In [16]:
h.loc[h.hownrent==2, 'hownrent'] = 0
h.hownrent.describe()

count    2.608019e+06
mean     5.869478e-01
std      4.923821e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: hownrent, dtype: float64

In [17]:
# Update column
orca.get_table('households').update_col_from_series('hownrent', h.hownrent)

# Check that it worked
orca.get_table('households').to_frame().hownrent.describe()

count    2.608019e+06
mean     5.869478e-01
std      4.923821e-01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: hownrent, dtype: float64

In [18]:
m = BinaryLogitStep()

In [20]:
m.tables = (['households','buildings'])

In [21]:
m.model_expression = 'hownrent ~ income + persons + stories'

In [23]:
m.filters = ['income < 500000']

In [27]:
m.fit()

Optimization terminated successfully.
         Current function value: 0.618479
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               hownrent   No. Observations:              2594607
Model:                          Logit   Df Residuals:                  2594603
Method:                           MLE   Df Model:                            3
Date:                Thu, 15 Mar 2018   Pseudo R-squ.:                 0.08808
Time:                        10:39:04   Log-Likelihood:            -1.6047e+06
converged:                       True   LL-Null:                   -1.7597e+06
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0793      0.004     19.676      0.000       0.071       0.087
income      1.133e-05    2.8e

In [28]:
m.out_column = 'taz'

In [29]:
orca.get_table('households').to_frame().taz.describe()

count    2.608019e+06
mean     7.526463e+02
std      4.301403e+02
min      1.000000e+00
25%      3.760000e+02
50%      7.630000e+02
75%      1.146000e+03
max      1.454000e+03
Name: taz, dtype: float64

In [30]:
m.run()

In [33]:
orca.get_table('households').to_frame().taz.describe()

count    2.608019e+06
mean     1.127045e+00
std      1.808491e+01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.252000e+03
Name: taz, dtype: float64

In [34]:
m.register()

In [35]:
orca.list_steps()

['test_automated_registration', 'test_manual_registration']

In [36]:
mm.list_steps()

[{'name': 'test_automated_registration', 'tags': [], 'type': 'RegressionStep'},
 {'name': 'BinaryLogitStep-20180315-103903',
  'tags': None,
  'type': 'BinaryLogitStep'}]

In [37]:
new = mm.get_step('BinaryLogitStep-20180315-103903')

KeyError: 'out_filters'

In [ ]:

orca.run(['name', 'name'])